In [3]:
from pathlib import Path
import pandas as pd
from lib import search_df
from IPython.display import Markdown as md

In [7]:
df = pd.read_csv(Path("data") / 'reflections.csv') 

def search_display(q: str, df: pd.DataFrame, limit: int = 25):
    result = search_df(q, df, limit)
    for i, row in result.iterrows():
        text = f"({str(row['result_distance'])[:6]}) {row['result_text']}"
        display(md(text))

In [5]:
students = df[df.first_gen_coll_student == 1]
search_df("athletics sports", students)

,perm,survey_50,HSGPA,student_selection,final_placement,auto_read,cwp_quarter,data_permission,language,PR_quarter,...,r4,EOPGRP,Gender,ETH_GRP,ETH,DEPT0,first_gen_coll_student,result_question_id,result_distance,result_text
0,Student2499,46.0,NaN,W2,W2,read,M2-22,My data collected as part of this project may ...,"I am comfortable using multiple languages, inc...",None,...,I believe this is the best option for me due t...,NOT_EOP,M,10,Foreign,Economics,1,r3,14.979328,"The subject ""stereotypes"" was mostly tackled d..."
1,Student1500,40.0,4.15,W1,W1,auto,M2-21,My data collected as part of this project may ...,"I am comfortable using multiple languages, inc...",None,...,I believe that starting with Writing 1 is the ...,EOP,M,2,Chicano,College of Letters and Science,1,r3,15.034551,These student writing samples are not very sim...
2,Student622,46.0,4.29,W2,W2,auto,F21,My data collected as part of this project may ...,"I am comfortable using multiple languages, inc...",None,...,Starting with Writing 2 fits me better because...,EOP,M,4,Asian,EEMB/MCDB,1,r1,15.227323,"In the summer of 2016, I embarked on my golf j..."
3,Student697,43.0,3.45,W2,W2,read,M1-21,My data collected as part of this project may ...,I am most comfortable using English.,None,...,"Up until this past year, English was my favour...",NOT_EOP,F,8,White,EEMB/MCDB,1,r2,15.318782,"Firstly, the ""Memory Project"" would be the eas..."
4,Student2149,46.0,4.07,W2,W2,auto,W23,My data collected as part of this project may ...,"I am comfortable using multiple languages, inc...",None,...,I believe writing 2 is a great fit for me beca...,EOP,M,2,Chicano,EEMB/MCDB,1,r2,15.326502,"For IB English my senior year, we had to write..."
5,Student1817,31.0,4.27,W1,W1,auto,M1-22,My data collected as part of this project may ...,I am most comfortable using English.,None,...,I believe I would like to start with writing o...,EOP,F,2,Chicano,Mechanical Engineering,1,r3,15.335045,None
6,Student2264,45.0,4.16,W2,W2,auto,M1-22,My data collected as part of this project may ...,I am most comfortable using English.,None,...,I think Writing 2 would be the best fit for me...,EOP,F,2,Chicano,Psychological and Brain Sciences,1,r3,15.457540,I've never written an essay about a scholarly ...
7,Student584,41.0,3.81,W2,W1,auto,F21,My data collected as part of this project may ...,I am most comfortable using English.,None,...,I believe this option best suits me because I ...,NOT_EOP,M,8,White,History,1,r3,15.734491,I don't have a lot of experience with these to...
8,Student697,43.0,3.45,W2,W2,read,M1-21,My data collected as part of this project may ...,I am most comfortable using English.,None,...,"Up until this past year, English was my favour...",NOT_EOP,F,8,White,EEMB/MCDB,1,r3,15.746632,I believe that the student writing samples are...
9,Student649,38.0,3.76,W1,W1,auto,M2-21,My data collected as part of this project may ...,I am most comfortable using English.,None,...,I think that writing 1 is the option of best f...,EOP,F,2,Chicano,Sociology,1,r3,15.893190,These students' writings are similar to when I...


In [6]:
from query import hello
hello()

ModuleNotFoundError: No module named 'query'